In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
import pandas as pd

s = Service("C:/path/to/chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://sena.territorio.la/index.php?login=true'
driver.get(url)

user='1095910391'
passwd='1095910391'

input_user=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div/form/table/tbody/tr[5]/td/input')
input_passwd=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div/form/table/tbody/tr[7]/td/label/input')

input_user.send_keys(user)
input_passwd.send_keys(passwd)

boton = driver.find_element(By.XPATH,'//*[@id="ingresar"]')
boton.click()

time.sleep(2)
course = driver.find_element(By.XPATH,'/html/body/div[12]/div/div[2]/div[1]/div[3]/div[2]/ul/li[1]/div/div[2]/span[1]/div/a')
course.click()

time.sleep(2)
evidencia = driver.find_element(By.XPATH,'/html/body/div[15]/table[1]/tbody/tr/td[1]/div/ul/li/ul/li[7]/a')
evidencia.click()
resultados = {}

for i in range(1, 165):  # Iterar N veces
    time.sleep(4)
    titulo = driver.find_element(By.XPATH, f'/html/body/div[15]/table[1]/tbody/tr/td[2]/div[3]/div[3]/div[1]/div[4]/div[{i}]/table/tbody/tr/td[1]/a').text
    time.sleep(4)
    calificacion = driver.find_element(By.XPATH, f'/html/body/div[15]/table[1]/tbody/tr/td[2]/div[3]/div[3]/div[1]/div[4]/div[{i}]/table/tbody/tr/td[4]/span').text
    resultados[titulo] = calificacion

df = pd.DataFrame(lista_resultados, columns=['Titulo', 'Calificacion'])


In [11]:
# Establecer el ancho de las columnas en 100
pd.set_option('display.max_colwidth', 500)

# Imprimir el dataframe completo
print(df)

                                                                                                                                     Titulo  \
0                                                          Ficha antropométrica de valoración de la condición física GA6-230101507-AA3-EV02   
1             Folleto de lesiones más comunes en el trabajo o vida cotidiana, y la importancia de las pausas activas GA6-230101507-AA4-EV01   
2                                                                      Infografía - Plan de higiene y gasto calórico GA6-230101507-AA2-EV01   
3                                                                         Módulos de software codificados y probados GA7-220501096-AA2-EV02   
4    Verificación de procedimientos para la definición de componentes front-end de la aplicación (Listas de chequeo) GA7-220501096-AA4-EV02   
..                                                                                                                                      ...   

In [16]:
df1 = df
# Extraer la parte del título que te interesa
df1['Titulo'] = df1['Titulo'].str.extract('(.+?)\sGA\d+-\w+-\w+')

# Crear la columna con la evidencia
df1['evidencia'] = df1['Titulo'].str.extract('(GA\d+-\w+-\w+)')

# Imprimir el dataframe resultante
print(df1)

                                                                                                              Titulo  \
0                                                          Ficha antropométrica de valoración de la condición física   
1             Folleto de lesiones más comunes en el trabajo o vida cotidiana, y la importancia de las pausas activas   
2                                                                      Infografía - Plan de higiene y gasto calórico   
3                                                                         Módulos de software codificados y probados   
4    Verificación de procedimientos para la definición de componentes front-end de la aplicación (Listas de chequeo)   
..                                                                                                               ...   
159                                                                                                              NaN   
160                                     

In [13]:
import pandas as pd

# Crear un dataframe de ejemplo
data = {'Titulo': ['Ficha antropométrica de valoración de la condición física GA6-230101507-AA3-EV02', 
                   'Folleto de lesiones más comunes en el trabajo', 
                   'Infografía - Plan de higiene y gasto calórico diario']}
df3= pd.DataFrame(data)

# Extraer la parte del título que te interesa
df3['Titulo_corto'] = df3['Titulo'].str.extract('(.+?)\sGA\d+-\w+-\w+')

# Crear la columna con la evidencia
df3['evidencia'] = df3['Titulo'].str.extract('(GA\d+-\w+-\w+)')

# Imprimir el dataframe resultante
print(df3)


                                                                             Titulo  \
0  Ficha antropométrica de valoración de la condición física GA6-230101507-AA3-EV02   
1                                     Folleto de lesiones más comunes en el trabajo   
2                              Infografía - Plan de higiene y gasto calórico diario   

                                                Titulo_corto  \
0  Ficha antropométrica de valoración de la condición física   
1                                                        NaN   
2                                                        NaN   

           evidencia  
0  GA6-230101507-AA3  
1                NaN  
2                NaN  


In [41]:
#postgressql
import psycopg2

# Conectarse a la base de datos
conn = psycopg2.connect(
    host="127.0.0.1",
    database="pcl",
    user="postgres",
    password="123456"
)
# Abrir un cursor
cur = conn.cursor()

# Iterar sobre el diccionario y guardar los datos en la base de datos
for titulo, calificacion in resultados.items():
    query = f"INSERT INTO adso.scraping_web (titulo, calificacion) VALUES ('{titulo}', '{calificacion}')"
    cur.execute(query)

# Guardar los cambios y cerrar la conexión
conn.commit()
cur.close()
conn.close()


In [ ]:
#Mysql
import mysql.connector

# Conectarse a la base de datos
cnx = mysql.connector.connect(
    host="tu_host",
    user="tu_usuario",
    password="tu_contraseña",
    database="tu_basedatos"
)

# Abrir un cursor
cursor = cnx.cursor()

# Iterar sobre el diccionario y guardar los datos en la base de datos
for titulo, calificacion in resultados.items():
    query = "INSERT INTO nombre_tabla (titulo, calificacion) VALUES (%s, %s)"
    data = (titulo, calificacion)
    cursor.execute(query, data)

# Guardar los cambios y cerrar la conexión
cnx.commit()
cursor.close()
cnx.close()
